# Addition of a Neural Network Model

After creating the previous three models for consideration, I decided to go back and take a look at the possibility of adding a fourth model. I wanted to see if I can use a model with Deep Neural Networks for a more realistic assessment of housing prices. I was able to reach my goal of coming under $200000, but I believe that I can do even better with the use of a neural network. I am going to bring in my cleaned data so that I can go right into the modeling process. 

# Importing Libraries

In [1]:
import keras
import numpy as np
import pandas as pd

# Bringing in the Cleaned Dataset

In [2]:
import pickle

#bringing in the cleaned
infile = open("Data/cleaned_data.pickle",'rb')
df = pickle.load(infile)
infile.close()

In [3]:
df.head()

,price,bedrooms,bathrooms,sqft_living,floors,waterfront,sqft_above,sqft_basement,sqft_living15,log_yard,...,zipcode_98146,zipcode_98148,zipcode_98155,zipcode_98166,zipcode_98168,zipcode_98177,zipcode_98178,zipcode_98188,zipcode_98198,zipcode_98199
0,365000.0,4.0,2.0,2070.0,2.0,0,2070.0,0,2390,8.969287,...,0,0,0,0,0,0,0,0,0,0
1,865000.0,5.0,3.0,2900.0,1.0,0,1830.0,1070,2370,8.250620,...,0,0,0,0,0,0,0,0,0,0
2,1038000.0,4.0,2.0,3770.0,2.0,0,3770.0,0,3710,9.105868,...,0,0,0,0,0,0,0,0,0,0
3,1490000.0,3.0,4.0,4560.0,2.0,0,4560.0,0,4050,9.419628,...,0,0,0,0,0,0,0,0,0,0
4,711000.0,3.0,2.0,2550.0,2.0,0,2550.0,0,2250,8.318986,...,0,0,0,0,0,0,0,0,0,0


# Preparing the Data

In [4]:
#dropping the dependent variable
dvariables = df.iloc[:, 1:]

#this here will become the SHAPE variable in the function below
number_of_columns = len(dvariables.columns)

#isolating the target variable
target = df.iloc[:,0]

In [5]:
#defining the metric for the model
def root_mean_squared_error(y_true, y_pred):
    """This is a helper function to allow root_mean_squared_error
    as the evaluation metric"""
    return keras.backend.sqrt(keras.backend.mean(keras.backend.square(y_pred - y_true))) 

In [6]:
def create_model(num_layers, shape, optimizer):
    """This function takes in the number of hidden layers, shape of the input, 
    and the optimizer to create a model ready to be fit."""
    model = keras.Sequential()
    model.add(keras.layers.Dense(shape, activation = 'relu', input_shape=(shape, )))
    for layer in range(num_layers):
        model.add(keras.layers.Dense(shape, activation = 'relu'))
    model.add(keras.layers.Dense(1))
    
    
    model.compile(optimizer, loss = root_mean_squared_error, metrics = [root_mean_squared_error])
    return model

In [7]:
model_1 = create_model(1, number_of_columns, 'adam')

In [8]:
#fitting the model
history = model_1.fit(dvariables, target, epochs = 50, validation_split=.2) #creating a small validation set to assess overfitting

Epoch 1/50
433/433 [==============================] - 0s 1ms/step - loss: 358511.7500 - root_mean_squared_error: 358420.5000 - val_loss: 243031.9531 - val_root_mean_squared_error: 241580.0781
Epoch 2/50
433/433 [==============================] - 0s 807us/step - loss: 249755.2656 - root_mean_squared_error: 249680.6719 - val_loss: 241825.1250 - val_root_mean_squared_error: 240344.7344
Epoch 3/50
433/433 [==============================] - 0s 764us/step - loss: 250330.2812 - root_mean_squared_error: 250055.3281 - val_loss: 241532.8281 - val_root_mean_squared_error: 240010.5312
Epoch 4/50
433/433 [==============================] - 0s 762us/step - loss: 249649.8281 - root_mean_squared_error: 249361.1406 - val_loss: 241427.0938 - val_root_mean_squared_error: 239971.1406
Epoch 5/50
433/433 [==============================] - 0s 783us/step - loss: 248564.9688 - root_mean_squared_error: 248502.2812 - val_loss: 241674.8438 - val_root_mean_squared_error: 240250.3281
Epoch 6/50
433/433 [============

433/433 [==============================] - 0s 756us/step - loss: 232029.9688 - root_mean_squared_error: 231909.3750 - val_loss: 225984.8125 - val_root_mean_squared_error: 224647.1562
Epoch 44/50
433/433 [==============================] - 0s 736us/step - loss: 229983.2031 - root_mean_squared_error: 230523.5781 - val_loss: 225366.6094 - val_root_mean_squared_error: 223973.2500
Epoch 45/50
433/433 [==============================] - 0s 740us/step - loss: 231491.3906 - root_mean_squared_error: 231304.5156 - val_loss: 224291.5938 - val_root_mean_squared_error: 222956.8438
Epoch 46/50
433/433 [==============================] - 0s 741us/step - loss: 231051.6562 - root_mean_squared_error: 231079.7812 - val_loss: 224602.1719 - val_root_mean_squared_error: 223292.3281
Epoch 47/50
433/433 [==============================] - 0s 755us/step - loss: 229666.3125 - root_mean_squared_error: 229524.8594 - val_loss: 223034.7969 - val_root_mean_squared_error: 221687.2969
Epoch 48/50
433/433 [================

# Evaluation

In [9]:
print('Evaluation at Epoch 50:\n')
for key in history.history:
    print(key + ':' + str(history.history[key][-1]) + '\n')

Evaluation at Epoch 50:

loss:227494.21875

root_mean_squared_error:227371.515625

val_loss:221076.796875

val_root_mean_squared_error:219760.671875



It is great to see that there has been no evidence of overfitting here. The validation set is proved to be superior due to the reduction of loss and error. In terms of the actual metric the model has not crossed the threshold of the goal RMSE. For now I am led to believe that the original chosen model shows the most promise.

# Second Model Attempt

For an attempt at improvement, I am going to try another model with more layers to see how that would compare to the first attempt. 

In [10]:
model_2 = create_model(3, number_of_columns, 'adam')

In [11]:
history_2 = model_2.fit(dvariables, target, epochs = 50, validation_split = .2)

Epoch 1/50
433/433 [==============================] - 1s 1ms/step - loss: 298888.3750 - root_mean_squared_error: 298651.1562 - val_loss: 241621.6562 - val_root_mean_squared_error: 240096.8750
Epoch 2/50
433/433 [==============================] - 0s 953us/step - loss: 249873.5938 - root_mean_squared_error: 249754.1406 - val_loss: 240968.4375 - val_root_mean_squared_error: 239437.3906
Epoch 3/50
433/433 [==============================] - 0s 902us/step - loss: 249397.2812 - root_mean_squared_error: 249295.0000 - val_loss: 245140.7500 - val_root_mean_squared_error: 243904.3594
Epoch 4/50
433/433 [==============================] - 0s 956us/step - loss: 248311.0625 - root_mean_squared_error: 248244.6406 - val_loss: 238911.0312 - val_root_mean_squared_error: 237481.7812
Epoch 5/50
433/433 [==============================] - 0s 924us/step - loss: 247032.5938 - root_mean_squared_error: 246830.4531 - val_loss: 237717.4375 - val_root_mean_squared_error: 236285.1250
Epoch 6/50
433/433 [============

433/433 [==============================] - 0s 900us/step - loss: 169450.2500 - root_mean_squared_error: 169348.1875 - val_loss: 160400.0625 - val_root_mean_squared_error: 159818.6406
Epoch 44/50
433/433 [==============================] - 0s 907us/step - loss: 170069.4531 - root_mean_squared_error: 169940.7344 - val_loss: 157313.7812 - val_root_mean_squared_error: 156923.0625
Epoch 45/50
433/433 [==============================] - 0s 912us/step - loss: 163766.5781 - root_mean_squared_error: 163610.3125 - val_loss: 150810.0938 - val_root_mean_squared_error: 150143.8125
Epoch 46/50
433/433 [==============================] - 0s 905us/step - loss: 168731.9219 - root_mean_squared_error: 168690.1875 - val_loss: 195450.5156 - val_root_mean_squared_error: 194128.1094
Epoch 47/50
433/433 [==============================] - 0s 901us/step - loss: 169017.5625 - root_mean_squared_error: 168930.6250 - val_loss: 158245.4844 - val_root_mean_squared_error: 157807.9219
Epoch 48/50
433/433 [================

In [12]:
print('Evaluation at Epoch 50:\n')
for key in history_2.history:
    print(key + ':' + str(history_2.history[key][-1]) + '\n')

Evaluation at Epoch 50:

loss:169516.265625

root_mean_squared_error:169379.328125

val_loss:144929.171875

val_root_mean_squared_error:144320.703125



The error here seemed to be reduced. This tells us that more layers were needed to get a better estimate of the target. I think another way to really zero in on an improvement would be to use Grid Search. That is the next step. 